In [1]:
## Main def function

import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)



def process_e1(a):
    e1 = []
    
    for sublist in a:
        for item in sublist:
            c = str(item).replace("(", "").replace(")", "").replace(",", "").split("\n")
            e1.append(c)
    
    return e1



def filter_digit_starting_items(e1):
   z = []

   for i in e1:
      s = "Levied"
      if not any(s in item for item in i):
         z.append(i)
   return z



# Function to remove the item before the string starting with '*** Exit Load Deducted' and store in separate list
def process_exit_load(data):
    removed_items = []
    for sublist in data:
        indices_to_remove = []
        for i in range(1, len(sublist)):
            if sublist[i].startswith('*** Exit Load Deducted'):
                indices_to_remove.append(i - 1)
        # Remove items in reverse order to avoid index shifting issues
        for index in sorted(indices_to_remove, reverse=True):
            removed_items.append(sublist[index])
            del sublist[index]
    return data



def cleaned_data(removed_items):

    b1 =[]
    for i in removed_items:
        for j in i:
            if j[:1].isdigit():
                b1.append(j)
    return b1



def date_filter(b1):

    date_patterns = [
        r'\b(0?[1-9]|[12][0-9]|3[01])[-\/](0?[1-9]|1[0-2])[-\/](\d{4})\b',  # D/M/YYYY or DD/MM/YYYY
        r'\b(0?[1-9]|1[0-2])[-\/](0?[1-9]|[12][0-9]|3[01])[-\/](\d{4})\b',  # M/D/YYYY or MM/DD/YYYY
        r'\b(\d{4})[-\/](0?[1-9]|1[0-2])[-\/](0?[1-9]|[12][0-9]|3[01])\b',  # YYYY/M/D or YYYY/MM/DD
        r'\b(January|February|March|April|May|June|July|August|September|October|November|December)\s(0?[1-9]|[12][0-9]|3[01]),\s(\d{4})\b',  # Month D, YYYY or Month DD, YYYY
        r'\b(0?[1-9]|[12][0-9]|3[01])\s(January|February|March|April|May|June|July|August|September|October|November|December)\s(\d{4})\b'  # D Month YYYY or DD Month YYYY
    ]

    combined_pattern = '|'.join(date_patterns)

    with_dates = []
    without_dates = []
    for item in b1:
        if re.search(combined_pattern, item):
            with_dates.append(item)
        else:
            without_dates.append(item)

        data = without_dates
    return with_dates, data




def create_groups(data, start_position):
    groups = []
    length = len(data)
    
    # Iterate through the list in steps of 5 starting from the start_position
    for i in range(start_position, length, 6):
        group = data[i:i + 6]  # Get the next 5 items
        groups.append(group)   # Append the group to the groups list
    
    return groups


# Function to convert groups to a format suitable for CSV writing
def groups_to_dict(groups):
    data_dicts = []
    for group in groups:
        data_dict = {
            'Balance_Unit': group[0],  # Replace 'field1', 'field2', etc., with actual field names
            'Amount(INR)': group[1],
            'NAV(INR)': group[2],
            'Price(INR)': group[3],
            'No_of_unit': group[4]
            # Add as many fields as necessary based on the structure of `group`
        }
        data_dicts.append(data_dict)
    return data_dicts


# def removed_lien(b):
#     re = []
#     n = []
#     for i in b:
#         for j in i:
#             # print(j)
#             if j.startswith("***Lien") or j.startswith("***Invalid") or j.startswith("***Multiple"):
#                 re.append(j)
#             else:
#                 n.append([j])
#     return n

def removed_lien(b):
    re = []
    n = []
    for i in a:
        # print(i)
        for j in i:
            # print(j)
            b = str(j)
            # print(type(b))
            if b.startswith("***Lien") or b.startswith("***Invalid") or b.startswith("***Multiple"):
                re.append(b)
            else:
                n.append([b])
    return n
            

def transaction_type(removed_items):
    t = []
    for i in removed_items:
        for j in i:
            if j.lower().startswith("redemption") :
                t.append(j)
            if j.lower().startswith("purchase"):
                t.append(j)
    return t

 

# Define regex patterns for each keyword
patterns = {
    "Folio number": r"pageFolio No.\s*([\d\s/]+)",}

# Function to extract information using regex patterns
def folio_number(text, patterns):
    extracted_data = []
    transactions = re.split(r'(?=Folio number)', text)
    
    for transaction in transactions:
        data = {}
        for key, pattern in patterns.items():
            result = re.search(pattern, transaction)
            data[key] = result.group(1).strip() if result else None
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
        extracted_numbers = [item['Folio number'] for item in extracted_data]

    return extracted_numbers


# def company(c):
#     com =[]
#     for i in c:
#         v = i.lower()
#         if v.endswith("ltd") or v.endswith("pvt ltd") or v.endswith("limited"):
#             com.append(i)
#     return com


from pypdf import PdfReader 

# def company_name_folio(file):
#     folio = []
#     co =[]
#     for i in os.listdir(file):
#         pdf_path = os.path.join(file,i)
#         reader = PdfReader(pdf_path)

#         page = reader.pages[0] 
        
#         # extracting text from page 
#         text = page.extract_text() 
#         folio1 = folio_number(text, patterns)
#         # f = folio(text,patterns)
#         folio.append(folio1)
#         # print(extracted_info)
#         a = (text.split('\n'))
#         c = a[0:10]
#         # comp = company(c)
#         # co.append(comp)
#         co.append(c[3])
#     return co,folio

# 
def insi_num(a):

    v = []
    for i in a:
        for j in i:
            c1 = str(j)
            c = (c1.split("/"))
            for k in c:
                if k.lower().startswith("inf") or k.lower().startswith(" inf"):
                    v.append(k)
    return v

def company_name(a):
    com = []
    for i in a[0:2]:
        for j in i:
            v = str(j).lower().split("\n")
            # print(v)
            for k in v:
                # print(k)
                if k.endswith("ltd") or k.endswith("pvt ltd") or k.endswith("limited"):
                    com.append(k)
    return com




predefined_string = "Amount, Transaction Type, NAV, Price, NO_of_Units, Balance_Units"


In [2]:
# main code skips pdf (no data)
import os
import re
import pandas as pd
import shutil
from PyPDF2 import PdfReader



# folio = []
patterns = "pageFolio No.\s*([\d\s/]+)" 
# # # patterns = r'(?:page|\d+)Folio\s*No\.\s*(\d{8,10}\s*/\s*\d{1,3})'
patterns1 = r'(?:page|\d+)Folio\s*No\.\s*(\d{8,10}\s*/\s*\d{1,3})'



file = r"D:\Python\ajit\ICICI\pdf_t_table"
# file = r"D:\Python\ajit\ICICI\prob" 
# file = r'D:\Python\ajit\text_extract\try2\try6\op1'


# output_folder = r"D:\Python\ajit\ICICI\prob_csv"  
output_folder = r"D:\Python\ajit\ICICI\table_to_csv"
# output_folder = r'D:\Python\ajit\text_extract\try2\try6\all_csv'

pdf_file = r"D:\Python\ajit\ICICI\ICICI"
# pdf_file = r"D:\Python\ajit\ICICI\prob_pdf"
# pdf_file = r'D:\Python\ajit\text_extract\try2\try6\new_pdf'

new_pdf_path = r"D:\Python\ajit\ICICI\moved_pdf"



# Initialize the folio list
folio = []

# Iterate over the Excel files
for i in os.listdir(file):
    xl_path = os.path.join(file, i)
    print(xl_path)
    csv_filename = f"{os.path.splitext(i)[0]}.csv"
    csv_path = os.path.join(output_folder, csv_filename)
    print(csv_path)
    
    # Check if the Excel file has more than 2 columns
    if xl_path.endswith("xlsx"):
        xls = pd.ExcelFile(xl_path)
        table_name = xls.sheet_names
        df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
        a1 = df1.values.tolist()
        c = company_name(a1)
        
        all_groups = []
        all_tran = []
        all_isni = []
        insi = []
        for j in table_name:
            print(j)
            df = pd.read_excel(xls, sheet_name=j, header=None)
            a = df.values.tolist()
            for l in a[0:2]:
                s = str(l)
                vector1 = text_to_vector(predefined_string)
                vector2 = text_to_vector(s)
                cosine = get_cosine(vector1, vector2)
                if cosine >= 0.07:
                    q = removed_lien(a)
                    v = process_e1(q)
                    z1 = filter_digit_starting_items(v)
                    removed_items = process_exit_load(z1)
                    b2 = cleaned_data(removed_items)
                    u = transaction_type(removed_items)
                    all_tran.extend(u)

                    ins = insi_num(a)
                    if len(u) >= 1:
                        insi.append(ins)

                    empty_list = [x for x in insi if x != []]
                    if len(u) < len(empty_list):
                        empty_list.pop(0)

                    m = v[0][0]
                    if m[:1].isdigit():
                        start_position = 2
                        print('start_position_0')
                        groups = create_groups(b2, start_position)
                    else:
                        start_position = 0
                        groups = create_groups(b2, start_position)
                        print('start_position_1')

                    all_groups.extend(groups)
                    all_isni.extend(empty_list * len(groups))
        
        # Create DataFrame using all_groups
        df = pd.DataFrame(all_groups)
        
        # Check if DataFrame has more than 2 columns
        if df.shape[1] > 2:
            # Load PDF file
            pdf_file_path = os.path.join(pdf_file, f"{os.path.splitext(i)[0]}.pdf")
            if os.path.exists(pdf_file_path):
                reader = PdfReader(pdf_file_path)
                page = reader.pages[0]
                text = page.extract_text()
                
                # Extract numbers from the page
                numbers = re.findall(patterns, text)
                if len(numbers) == 0:
                    folio1 = re.findall(patterns1, text)
                    folio.append(folio1)
                else:
                    folio.append(numbers)
                
                # Create final DataFrame
                df.columns = ['Amount (INR)', 'NAV(INR)', 'Price(INR)', 'No_of_Unit', 'Balance Limit', 'Date']
                df['Company Name'] = pd.Series([c[0]] * len(df), index=df.index)
                df['Folio Number'] = pd.Series([folio[0][0]] * len(df), index=df.index)
                df['Transaction Type'] = pd.Series(all_tran)
                df['ISIN Number'] = pd.Series(all_isni)
                c.pop(0)
                folio.pop(0)
                print(df)
                # df.to_csv(csv_path)
                
                # Move PDF file to a new folder based on Excel file path
                new_folder = os.path.join(output_folder, "processed_pdfs")
                if not os.path.exists(new_folder):
                    os.makedirs(new_folder)
                new_pdf_path = os.path.join(new_folder, f"{os.path.splitext(i)[0]}.pdf")
                shutil.move(pdf_file_path, new_pdf_path)
                print(f"Moved PDF file to: {new_pdf_path}")

D:\Python\ajit\ICICI\pdf_t_table\10352635.xlsx
D:\Python\ajit\ICICI\table_to_csv\10352635.csv
Page 1 - Table 1
Page 1 - Table 2
Page 1 - Table 3
Page 1 - Table 4
start_position_0
Page 1 - Table 5
start_position_0
Page 2 - Table 1
Page 2 - Table 2
start_position_0
Page 2 - Table 3
  Amount (INR)   NAV(INR) Price(INR) No_of_Unit Balance Limit        Date  \
0    999950.00  1320.3240  1320.3240    757.352    103529.543  07/08/2024   

                 Company Name   Folio Number   Transaction Type  \
0  sipat transmission limited  10352635 / 40  Purchase - DIRECT   

        ISIN Number  
0  [ INF109KC11G1 ]  
Moved PDF file to: D:\Python\ajit\ICICI\table_to_csv\processed_pdfs\10352635.pdf
D:\Python\ajit\ICICI\pdf_t_table\10352655.xlsx
D:\Python\ajit\ICICI\table_to_csv\10352655.csv
Page 1 - Table 1
Page 1 - Table 2
Page 1 - Table 3
Page 1 - Table 4
start_position_0
Page 1 - Table 5
start_position_0
Page 2 - Table 1
Page 2 - Table 2
start_position_0
Page 2 - Table 3
   Amount (INR)   NAV(I